----

# 남북한 뉴스기사 전처리
작성자: 김지민, 김영성

이 노트북은 북한(KCNA)과 남한(국방일보) 기사 데이터를 전처리하여 분석 및 시각화를 위한 정제된 CSV 파일로 저장하는 과정 전체를 다룹니다.

## 주요 내용
- 특수 문자 및 날짜 표현 정제를 위한 텍스트 정제 함수 정의 (`clean_content`)
- 기사 제목 및 본문 데이터 정제
- 남한 기사 데이터 전처리 (`preprocess_south_article`)
  - 국가별 / 전체 기사로 구분 저장
- 북한 기사 데이터 전처리 (`preprocess_north_article`)
  - 필요 없는 컬럼 삭제 및 컬럼명 통일
  - 국가별 / 전체 기사로 구분 저장

----

In [9]:
import re
import os
import glob
import pandas as pd

----

## 1. 텍스트 정제 함수 (`clean_content`)
- 날짜 관련 문자('월', '일', ',') 제거
- 하이픈(-), 줄바꿈(\n), 괄호((), 《》) 제거
- 불필요한 공백 제거 (양쪽 strip)
- 주로 기사 Title, Content 전처리에 사용

In [10]:
def clean_content(text):
    text = str(text)
    text = re.sub(r'[월일,]', '', text)       
    text = re.sub(r'-', '', text)               
    text = re.sub(r'\n', '', text)             
    text = re.sub(r'[()]', '', text)        
    text = re.sub(r'[《》]', '', text)    
    return text.strip()

-------

## 2. 남한 기사 전처리
### 1) 남한 기사 정제 함수 (`preprocess_south_article`)
- Date 컬럼을 datetime 형식으로 변환한 후 "YYYY-MM-DD" 문자열로 재포맷
- `Category` 컬럼을 `Title`에서 추출 (`<...>` 안의 내용)
- `Title` 컬럼에서 `<...>` 부분 제거
- 필요시 Country 컬럼 추가
- `content` → `Content`로 컬럼명 정리
- `Title`, `Content` 컬럼에 `clean_content()` 적용 (불필요한 문자 제거)
- 지정 폴더(`save_dir`)에 `"i_한국기사_{국가}.csv"` 또는 `"i_한국기사_전체.csv"`로 저장

In [6]:
def preprocess_south_article(file, i, country, save_dir):
    df = pd.read_csv(file)

    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    df = df.sort_values(by='Date', ascending=True).reset_index(drop=True)

    df['Category'] = df['Title'].str.extract(r'<([^<>]+)>$')
    df['Title'] = df['Title'].str.replace(r'\s*<[^<>]+>$', '', regex=True)

    if country:
        df['Country'] = country

    df = df.rename(columns={'content': 'Content'})

    for col in ['Content', 'Title']:
        if col in df.columns:
            df[col] = df[col].apply(clean_content)

    save_name = f"{i}_남한기사_{country if country else '전체'}.csv"
    df.to_csv(f"{save_dir}/{save_name}", index=False)

### 2) 국가별 남한 기사 전처리

In [ ]:
for i in range(1, 16):
    for country in ['러시아', '미국', '북한', '중국']:
        preprocess_south_article(
            file=f'{i}_남한기사_{country}.csv',
            i=i,
            country=country,
            save_dir='국가별'
        )

### 3) 전체 남한 기사 전처리

In [22]:
for i in range(1, 16):
    preprocess_south_article(
        file=f'{i}_남한기사_전체.csv',
        i=i,
        country=None,
        save_dir='전체'
    )

-------

# 3. 북한 기사 전처리
### 1) 북한 기사 CSV 파일 정제 및 저장 함수 (`preprocess_article`)
- Date 컬럼을 datetime 형식으로 변환한 후 "YYYY-MM-DD" 문자열로 재포맷
- 필요시 Country 컬럼 추가
- 필요시 컬럼 삭제(drop_columns) 및 이름 변경(rename_columns)
- Title, Content 컬럼에 대해 clean_content() 함수 적용 (특수문자 제거 등 텍스트 정제)
- 지정된 폴더(save_dir)에 "i_북한기사_{국가}.csv" 또는 "i_북한기사_전체.csv"로 저장

In [12]:
def preprocess_north_article(file, i, country, save_dir, drop_columns, rename_columns):
    df = pd.read_csv(file)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    df = df.sort_values(by='Date', ascending=True).reset_index(drop=True)

    if country:
        df['Country'] = country

    if drop_columns:
        df = df.drop(columns=drop_columns, errors='ignore')
    
    if rename_columns:
        df = df.rename(columns=rename_columns)
    
    for col in ['Content', 'Title']:
        if col in df.columns:
            df[col] = df[col].apply(clean_content)
    
    save_name = f"{i}_북한기사_{country if country else '전체'}.csv"
    df.to_csv(f"{save_dir}/{save_name}", index=False)

### 2) 국가별 북한 기사 전처리

In [ ]:
for i in range(1, 16):
    for country in ['러시아', '미국', '중국', '한국']:
        preprocess_north_article(
            file=f'{i}_북한기사_{country}.csv', 
            i=i, 
            country=country, 
            save_dir='국가별',
            drop_columns=None, 
            rename_columns=None
        )

### 3) 전체 북한 기사 전처리

In [ ]:
for i in range(1, 16):
    preprocess_north_article(
        file=f'{i}_북한기사_전체.csv', 
        i=i, 
        country=None, 
        save_dir='전체', 
        drop_columns=['Content'], 
        rename_columns={'content': 'Content'}
    )

-----------